<a href="https://colab.research.google.com/github/rencete/computer-vision/blob/CIFAR-10/CIFAR-10/CIFAR_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import libraries

In [1]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model

import numpy as np

In [2]:
# Constants
MODEL_FILENAME_N_PATH='cifar10_model.h5'
MODEL_FIT_VERBOSITY=0
MODEL_EVALUATE_VERBOSITY=0
TEST_IMAGE_FILENAME_N_PATH='sample.png'

In [3]:
# load train and test dataset
def load_dataset():
	# load dataset
	(trainX, trainY), (testX, testY) = cifar10.load_data()
 
  # one hot encode target values
	trainY = to_categorical(trainY)
	testY = to_categorical(testY)
	return trainX, trainY, testX, testY

In [4]:
# scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
 
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
  
	# return normalized images
	return train_norm, test_norm

In [5]:
# define cnn model
def define_model():
  model = Sequential()
 
  # Add details for hidden layer 1
  model.add(Conv2D(32, (3, 3), padding='same', activation='relu', 
                   kernel_initializer='he_uniform', input_shape=(32, 32, 3)))
  model.add(Conv2D(32, (3, 3), padding='same', activation='relu', 
                   kernel_initializer='he_uniform'))
  model.add(MaxPooling2D((2, 2)))

  # Add details for hidden layer 2
  model.add(Conv2D(64, (3, 3), padding='same', activation='relu', 
                   kernel_initializer='he_uniform'))
  model.add(Conv2D(64, (3, 3), padding='same', activation='relu', 
                   kernel_initializer='he_uniform'))
  model.add(MaxPooling2D((2, 2)))

  # Add details for hidden layer 3
  model.add(Conv2D(128, (3, 3), padding='same', activation='relu', 
                   kernel_initializer='he_uniform'))
  model.add(Conv2D(128, (3, 3), padding='same', activation='relu', 
                   kernel_initializer='he_uniform'))
  model.add(MaxPooling2D((2, 2)))

  # Flatten before connecting to fully interconnected layer for classification
  model.add(Flatten())

  # Fully connected layer
  model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))

  # Output layer
  model.add(Dense(10, activation='softmax'))
  
  # compile model
  opt = SGD(learning_rate=0.01, momentum=0.9)
  model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
  return model

In [6]:
def train_model(model, X, y):
  # fit model
  model.fit(X, y, epochs=10, batch_size=32, verbose=MODEL_FIT_VERBOSITY)
  # save model
  model.save(MODEL_FILENAME_N_PATH)

In [7]:
def evaluate_model(testX, testY):
  # Load the saved model
  model = load_model(MODEL_FILENAME_N_PATH)

  # evaluate model on test dataset
  loss, acc = model.evaluate(testX, testY, verbose=MODEL_EVALUATE_VERBOSITY)
  
  print("="*80)
  print("Model evaluation results as follows:")
  print('Loss: %.3f' % (loss * 100.0))
  print('Accuracy: %.3f' % (acc * 100.0))
  print("="*80)

In [8]:
def load_image(filename):
  #load the image
  img = load_img(filename, color_mode='rgb', target_size=(32, 32))

  #convert to array
  img = img_to_array(img)

  #reshape into a single sample
  img = img.reshape(1, 32, 32, 3)

  #prepare pixel data
  img = img.astype('float32')
  img = img / 255.0
  return img

In [9]:
def name_prediction(value):
  names = [
           'airplane',
           'automobile',
           'bird',
           'cat',
           'deer',
           'dog',
           'frog',
           'horse',
           'ship',
           'truck'
           ]
  return names[value]

In [10]:
def predict_sample():
  # Load the sample image for prediction
  img = load_image(TEST_IMAGE_FILENAME_N_PATH)

  # Load the saved model
  model = load_model(MODEL_FILENAME_N_PATH)

  # predict the classification
  result = model.predict(img)

  prediction = name_prediction(np.argmax(result))
  print("The sample is predicted to be a(n)", prediction)

In [11]:
# run the test harness for evaluating a model
def run_test_harness():
  # load dataset
  trainX, trainY, testX, testY = load_dataset()
  # prepare pixel data
  trainX, testX = prep_pixels(trainX, testX)
  # define model
  model = define_model()

  # train and save trained model
  train_model(model, trainX, trainY)

  # evaluate model performance
  evaluate_model(testX, testY)

  # test model by making a prediction
  predict_sample()

In [12]:
# entry point, run the test harness
run_test_harness()

Model evaluation results as follows:
Loss: 85.765
Accuracy: 74.180
The sample is predicted to be a(n) deer
